# 借助图数据库进行关联查询

## CHPO可视化

### 在特定表型分类下，查询不同表型之间的关联
由于实体间的关系是 ***Phenotypic->PhenotypicCategory->Term*** , 所以对Term来说要查询表型, 需要递归 **in** 关系的节点, 通过 **repeat** 操作可重复查询 **in** 节点，再通过 **until** 限定终止条件为:
+ 表型的中文名为['脑肉芽肿','肺炎','酒后面部潮红','牙龈出血']四者之一<br/>

从表型来看四者分别描述的人体不同部位，但从查询结果来看四者之间有一定关联，且相关性有差异，如 **"牙龈出血"** 和 **"脑肉芽肿"** 都属于 **"血液和造血组织异常"** 分类，相关性更近。

In [1]:
%%gremlin -p v,v,v
g.V("表型异常").hasLabel('Term')
.repeat(
    in()
)
.until(
    has('name_cn',within('脑肉芽肿','肺炎','酒后面部潮红','牙龈出血'))
)
.simplepath()
.by(id)
.by(id)
.by('name_cn')


### 查询不同表型之间的关联
表型分类与表型之间存在着多对多的关系，所以从已知表型出发查询关系也是有必要的，下面的示例是从表型出发，分别查找不同表型所属的分类。
从结果来看:
+ **结膜炎** 、**结膜血管迂曲** 、**视网膜萎缩** 均与 **眼睛的异常** 相关
+ **结膜炎** 、**结膜血管迂曲** 均和 **头或颈部异常** 相关
+ **结膜炎** 、**结膜血管迂曲** 分别和 **免疫系统异常** 、 **心血管系统异常** 相关

In [2]:
%%gremlin -p v,v,v
g.V().hasLabel('Phenotypic').has('name_cn',within('结膜血管迂曲','结膜炎','视网膜萎缩'))
.repeat(
    out()
)
.until(hasLabel('PhenotypicCategory'))
.simplepath()
.by('name_cn')
.by(id)
.by(id)

// 从“结膜血管迂曲”表型出发，查询该表型关联的表型分类

In [8]:
%%gremlin -p v
g.V("遗传模式").hasLabel('PhenotypicCategory').in().values("name_cn")

In [59]:
%%gremlin
g.V().count()

In [62]:
%%gremlin
g.E().hasLabel("gene_to_phenotypes").count()